# Wikipedia Notable Life Expectancies
# [Notebook  13: Data Cleaning Part 12](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean12_thanak_2022_08_03.ipynb)
### Context

The
### Objective

The
### Data Dictionary
- Feature: Description

### Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### [Reading](), Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean11.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean11", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 98056 rows and 38 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,3.091042,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None,None,2.564949,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
98054,9,Aamir Liaquat Hussain,", 50, Pakistani journalist and politician, MNA .",https://en.wikipedia.org/wiki/Aamir_Liaquat_Hussain,99,2022,June,", since",None,,MNA,None,None,None,None,None,None,None,None,50.0,None,Pakistan,None,"2002 2007, since 2018",4.605170,0,0,0,0,0,1,0,0,1,0,0,0,2
98055,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,,member of the Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None,None,1.386294,1,0,0,0,0,0,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
97227,15,Bilquis Edhi,", 74, Pakistani nurse and humanitarian, heart failure.",https://en.wikipedia.org/wiki/Bilquis_Edhi,13,2022,April,None,None,nurse,heart failure,None,None,None,None,None,None,None,None,74.0,None,Pakistan,None,None,2.639057,0,1,0,0,0,0,0,0,0,0,0,0,1
87248,30,Sophie,", 34, Scottish singer-songwriter and record producer, accidental fall.",https://en.wikipedia.org/wiki/Sophie_(musician),94,2021,January,"""Bipp"", ""Lemonade""",None,,accidental fall,None,None,None,None,None,None,None,None,34.0,None,Scotland,None,"""Bipp"", ""Lemonade""",4.553877,0,0,0,0,0,1,0,0,0,0,0,0,1
58906,29,Elechi Amadi,", 82, Nigerian writer.",https://en.wikipedia.org/wiki/Elechi_Amadi,26,2016,June,None,None,,None,None,None,None,None,None,None,None,None,82.0,None,Nigeria,None,None,3.295837,0,0,0,0,0,1,0,0,0,0,0,0,1
23144,6,"Elward Thomas Brady, Jr.",", 60, American businessman and politician.","https://en.wikipedia.org/wiki/Elward_Thomas_Brady,_Jr.",6,2007,April,None,None,,None,None,None,None,None,None,None,None,None,60.0,None,United States of America,None,None,1.945910,0,0,0,0,1,0,0,0,1,0,0,0,2
32597,26,Gloria Stuart,", 100, American film actress , respiratory failure.",https://en.wikipedia.org/wiki/Gloria_Stuart,92,2010,September,",",None,,respiratory failure,None,None,None,None,None,None,None,None,100.0,None,United States of America,None,",",4.532599,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98056 entries, 0 to 98055
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   day                        98056 non-null  object 
 1   name                       98056 non-null  object 
 2   info                       98056 non-null  object 
 3   link                       98056 non-null  object 
 4   num_references             98056 non-null  int64  
 5   year                       98056 non-null  int64  
 6   month                      98056 non-null  object 
 7   info_parenth               36661 non-null  object 
 8   info_1                     22 non-null     object 
 9   info_2                     98024 non-null  object 
 10  info_3                     48896 non-null  object 
 11  info_4                     10264 non-null  object 
 12  info_5                     1265 non-null   object 
 13  info_6                     181 non-null    obj

<IPython.core.display.Javascript object>

#### Observations:
- With our dataset loaded, we can pick up where we left off with extracting known_for values by rebuilding `known_for_dict`.

### Extracting `known_for` Continued

#### Finding `known_for` Roles in `info_2`

In [6]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [7]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [8]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "religion" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [9]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [10]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "economics editor"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [201]:
# Creating lists for each category
politics_govt_law = [
    "revolutionary socialist and workers' leader",
    "' leader",  # before business_farming
    "Tlingit elder",
    "Governor of Benue State",
    "veterans advocate",
    "traditional ruler of Ife",
    "ruler of Ras al Khaimah",
    "traditional ruler",
    "ruler of the",
    "ruler",
]

arts = [
    "graphic designer and pioneer in the field of computer graphics",
    "photographic director and videographer",
    "illustrator and graphic designer",
    "graphic designer and typographer",
    "television graphic designer",
    "graphic and type designer",
    "pornographic performer",
    "graphic designer",
    "gay pornographic",
    "pornographic",
    "graphic and",
    "graphic",
    "theatre and opera director and stage designer",
    "theatre director and voice coach",
    "theatre and opera administrator",
    "television and theatre director",
    "and Broadway theatre performer",
    "humorist and theatre director",
    "theatre director and designer",
    "theatre director and theorist",
    "theatre and concert director",
    "opera and theatre director",
    "theatre owner and manager",
    "theatre director and",
    "theatre impresario",
    "theatre publicist",
    "theatre director",
    "theatre designer",
    "theatre promoter",
    "theatre and",
    "theatre",
    "celebrity chef and television personality",
    "pastry chef and television personality",
    "chef and reality show contestant",
    "chef and television personality",
    "Michelin Star winning chef",
    "pioneering television chef",
    "Cajun chef and humorist",
    "famed New Orleans chef",
    "chef and gastronomist",
    "television chef",
    "celebrity chef",
    "internet chef",
    "pastry chef",
    "Creole chef",
    "music  chef",
    "head chef",
    "chef and",
    "chef",
    "stunt performer",
    "movie stuntman",
    "car customizer",
    "customizer",
    "stuntwoman",
    "BBC disc jockey and guru of the independent music scene",  # before sports
    "radio disc jockey and proponent of Pinoy rock",
    "Hall of Fame disc jockey and television host",
    "disc jockey and television personality",
    "disk jockey and sound system operator",
    "disc jockey and music news reporter",
    "disc jockey and record collector",
    "disk jockey known as 'Nightbird'",
    "disc jockey and television host",
    "game show host and disc jockey",
    "former BBC Radio disc jockey",
    "country music disc jockey",
    "BBC Radio disc jockey",
    "footwork disc jockey",
    "reggae disc jockey",
    "radio disc jockey",
    "radio disk jockey",
    "disc jockey and",
    "and disc jockey",
    "disc jockey",
    "disk jockey",
    "children book and magazine illustrator",
    "conceptual designer and illustrator",
    "illustrator for the original books",
    "concept designer and illustrator",
    "printmaker and book illustrator",
    "photo essayist and illustrator",
    "illustrator of children books",
    "illustrator and watercolorist",
    "caricaturist and illustrator",
    "illustrator and caricaturist",
    "science fiction illustrator",
    "magazine cover illustrator",
    "printmaker and illustrator",
    "children book illustrator",
    "illustrator and designer",
    "commercial illustrator",
    "comic book illustrator",
    "botanical illustrator",
    "children illustrator",
    "fashion illustrator",
    "fantasy illustrator",
    "comics illustrator",
    "manga illustrator",
    "comic illustrator",
    "book illustrator",
    "bird illustrator",
    "and illustrator",
    "illustrator and",
    "illustrator",
    "nurseryman",  # before sciences
    "correspondent and editor for United Press International",
    "editor in chief of King Features Syndicate",
    "Pulitzer Prize winning newspaper editor",
    "comic book and pulp magazine editor",
    "newspaper editor of the from until",
    "United Press International editor",
    "founding editor of stomach cancer",
    "wood carver and magazine editor",
    "sound designer and sound editor",
    "science fiction fanzine editor",
    "magazine and newspaper editor",
    "editorial page editor for the",
    "editor of black publications",
    "founding editor of magazine",
    "editor in chief of magazine",
    "Oscar winning sound editor",
    "newspaper editor in chief",
    "Composer and music editor",
    "sound designer and editor",
    "book and magazine editor",
    "science fiction editor",
    "newspaper chief editor",
    "photojournalism editor",
    "visual effects editor",
    "women magazine editor",
    "games magazine editor",
    "and newspaper editor",
    "mystery novel editor",
    "secretary and editor",
    "Disney comics editor",
    "and magazine editor",
    "book review editor",
    "managing editor of",
    "comic book editor",
    "publishing editor",
    "comic  and editor",
    "photo editor and",
    "newspaper editor",
    "magazine editor",
    "literary editor",
    "editor in chief",
    "fashion editor",
    "fiction editor",
    "sound editor",
    "photo editor",
    "music editor",
    "book editor",
    "news editor",
    "CNET editor",
    "and editor",
    "editor and",
    "editor of",
    "editor",
]
sports = [
    "professional road bicycle racer who won two stages of the Tour de",
    "Grand Prix motorcycle and short circuit road racer",
    "short circuit motorcycle road racer",
    "sport sailor and maxi yacht racer",
    "Grand Prix motorcycle road racer",
    "motorcycle and touring car racer",
    "professional road bicycle racer",
    "motor racer and IndyCar driver",
    "motorcycle sidecar road racer",
    "Hall of Fame motorcycle racer",
    "jet car driver and drag racer",
    "automobile racer and designer",
    "professional motocross racer",
    "motorcycle builder and racer",
    "Grand Prix motorcycle racer",
    "Paralympic wheelchair racer",
    "motorcycle speedway racer",
    "drag racer and crew chief",
    "racer and television host",
    "off road motorcycle racer",
    "motorcycle and auto racer",
    "Hall of Fame drag racer",
    "motorcycle rally racer",
    "motorcycle road racer",
    "Moto motorcycle racer",
    "land speed racer and",
    "motorcycle racer and",
    "motorcycle racer and",
    "motorcross racer and",
    "horse harness racer",
    "mountain bike racer",
    "road bicycle racer",
    "disabled ski racer",
    "hillclimbing racer",
    "powerboating racer",
    "cyclo cross racer",
    "touring car racer",
    "motorcycle racer",
    "alpine ski racer",
    "automobile racer",
    "wheelchair racer",
    "Alpine ski racer",
    "sprint car racer",
    "motocross racer",
    "stock car racer",
    "motorbike racer",
    "motorboat racer",
    "ski cross racer",
    "NHRA drag racer",
    "off road racer",
    "bicycle racer",
    "sidecar racer",
    "Air racer and",
    "barrel racer",
    "MotoGP racer",
    "yacht racer",
    "motor racer",
    "rally racer",
    "drag racer",
    "auto racer",
    "air racer",
    "ski racer",
    "and racer",
    "racer",
    "female jockey and pioneer in thoroughbred horse racing",
    "jockey and first woman in to receive a jockey licence",
    "National Hunt jockey and horse trainer",
    "race horse trainer and jockey mentor",
    "jockey in thoroughbred horse racing",
    "jockey in thoroughbred racing",
    "horse trainer and jockey",
    "jockey and horse trainer",
    "National Champion jockey",
    "National Hunt jockey",
    "horse racing jockey",
    "Hall of Fame jockey",
    "jockey and trainer",
    "jockey",
]
sciences = [
    "paleontologist and ornithologist",
    "ichthyologist and ornithologist",
    "ornithologist and",
    "and ornithologist",
    "ornithologist",
    "zoologist and advocate of evolutionary epistemology",
    "zoologist and neurophysiologist",
    "palaeontologist and zoologist",
    "paleontologist and zoologist",
    "soil zoologist and ecologist",
    "immunologist and zoologist",
    "zoologist and ecologist",
    "invertebrate zoologist",
    "zoologist  science",
    "medical zoologist",
    "turtle zoologist",
    "cryptozoologist",
    "and zoologist",
    "zoologist and",
    "zoologist",
    "healthcare advocate and registered nurse",
    "first nurse to earn a master degree",
    "nurse and nurse researcher",
    "nurse and nurse tutor",
    "mental health nurse",
    "nurse and nursing",
    "registered nurse",
    "Navy nurse",
    "Army nurse",
    "nurse and",
    "and nurse",
    "nurse",
    "endocrinologist and medical researcher",
    "pediatrician and medical researcher",
    "immunologist and medical researcher",
    "neurologist and medical researcher",
    "medical researcher in immunology",
    "biomedical researcher",
    "medical researcher",
]

business_farming = [
    "hotelier and casino owner",
    "hotelier and retailer",
    "hotelier and",
    "hotelier",
    "potato farmer and long distance runner",
    "farmer and landowner",
    "farmer and lobbyist",
    "rice farmer",
    "and farmer",
    "farmers'",
    "farmer",
]

academia_humanities = [
    "and Broadway theatre preservationist",  # before arts
]
law_enf_military_operator = [
    "colonel in the Army and of the most decorated women in military history",
    "Army General who commanded military operations in the War from to",
    "paramilitary intelligence chief and clandestine agent",
    "National Liberation Army paramilitary leader",
    "Sandinista guerrilla and military leader",
    "scientific military intelligence expert",
    "Prime Minister of and military leader",
    "Northern Alliance military commander",
    "Serb military commander in the War",
    "military communications listener",
    "military commander and warlord",
    "republican paramilitary leader",
    "military and security official",
    "head of military intelligence",
    "warlord and military figure",
    "Serb paramilitary commander",
    "Biafran military commander",
    "Chetnik military commander",
    "Hutu paramilitary leader",
    "ISIL military commander",
    "and paramilitary leader",
    "and military commander",
    "paramilitary commander",
    "military commander and",
    "era military commander",
    "military intelligence",
    "loyalist paramilitary",
    "military interpreter",
    "military veteran and",
    "military leader and",
    "paramilitary leader",
    "marine and military",
    "military commander",
    "military  designer",
    "military official",
    "military veteran",
    "and paramilitary",
    "military leader",
    "military office",
    "Hamas military",
    "military chief",
    "paramilitary",
    "military man",
    "military",
]

spiritual = ["religion and apologetics", "of religion", "religion"]
social = [
    "animal welfare campaigner",
    "social worker and Righteous Among the Nations",
    "youth social worker",
    "and social worker",
    "social worker",
    "charity fundraiser",
]
crime = ["and forger", "forger", "drug lord"]
event_record_other = [
    "chef and construction worker",  # before arts
    "construction worker",
    "ebola survivor",
    "anthrax attack victim",
]
other_species = []

<IPython.core.display.Javascript object>

In [12]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [13]:
# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Hideo_Ogata"].index
df.loc[index, "cause_of_death"] = "stomach cancer"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [14]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "politics_govt_law": politics_govt_law,
    "business_farming": business_farming,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sciences": sciences,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [15]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 3min 1s
Wall time: 3min 1s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
45373,30,Michael Palmer,", 71, American novelist, heart attack and stroke.",https://en.wikipedia.org/wiki/Michael_Palmer_(novelist),5,2013,October,None,None,,heart attack and stroke,None,None,None,None,None,None,None,None,71.0,None,United States of America,None,None,1.791759,0,0,0,0,0,1,0,0,0,0,0,0,1
49412,18,Gordon Faber,", 83, American politician.",https://en.wikipedia.org/wiki/Gordon_Faber,24,2014,August,None,None,,None,None,None,None,None,None,None,None,None,83.0,None,United States of America,None,None,3.218876,0,0,0,0,0,0,0,0,1,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [16]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 9762 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [17]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [18]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [19]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Jesuit" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [20]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [21]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "press director"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [22]:
# Creating lists for each category
politics_govt_law = [
    "managing director of the Abu Dhabi Investment Authority",  # before arts
    "director of Office of Telecommunications Policy",
    "director of the Mint from to",
    "Native advocate",
    "Peace",
]

arts = [
    "director of most episodes of Monty Python Flying Circus",
    "theater director who staged plays on and off Broadway",
    "publishing director of Burke Peerage Limited",
    "opera director and set and costume designer",
    "theatrical director and opera librettist",
    "director of John H Johnson Fashion Fair",
    "music director and music group founder",
    "cameraman and director of photography",
    "director and lyricist in the language",
    "operatic baritone and opera director",
    "television and music video director",
    "opera director and administrator",
    "festival director and cover girl",
    "managing director of BBC Radio",
    "organist and musical director",
    "television and radio director",
    "organist and choral director",
    "rock and roll tour director",
    "Emmy Award winning director",
    "short  and casting director",
    "theater and opera director",
    "opera director and manager",
    "opera director and hazzan",
    "public relations director",
    "theater director and mime",
    "radio program director",
    "marching band director",
    "college band director",
    "music video director",
    "operatic ic director",
    "commercial director",
    "theatrical director",
    "assistant director",
    "festival director",
    "Broadway director",
    "and news director",
    "director of Radio",
    "theater director",
    "casting director",
    "company director",
    "B movie director",
    "gallery director",
    "screen director",
    "choral director",
    "design director",
    "opera director",
    "music director",
    "movie director",
    "media director",
    "anime director",
    "radio director",
    "voice director",
    "press director",
    "band director",
    "set director",
    "director and",
    "director",
    "organist of the St Peter Basilica in Rome",
    "harpsichordist and organist",
    "organist and harpsichordist",
    "chorister and organist",
    "classical organist and",
    "cantor and organist",
    "cathedral organist",
    "classical organist",
    "concert organist",
    "stadium organist",
    "organist and",
    "organist",
]
sports = [
    "track and field coach and athletic director",  # before arts
    "manager and director in the Football League",
    "collegiate athletic director",
    "college athletic director",
    "runner and race director",
    "athletic director",
    "sporting director",
]
sciences = [
    "NASA mission director",  # before arts
]

business_farming = [
    "toy manufacturer and managing director of Lego",  # before arts
    "managing director of Ulsterbus and Citybus",
    "business director",
    "funeral director",
]
academia_humanities = [
    "deputy director of the National Air and Space Museum",  # before arts
    "director of the National Gallery of Art from to",
    "director of the Metropolitan Museum of Art",
    "director of the Cleveland Museum of Art",
    "oral history archive director",
    "curator and museum director",
    "library director",
    "museum director",
]
law_enf_military_operator = [
    "Army Lieutenant General and director of the National Security Agency",  # before arts
    "first director of the Coast Guard Women Reserve",
    "former CIA director",
    "director of the FBI",
]
spiritual = [
    "church music director and concert",  # before arts
    "Jesuit monk and",
    "Jesuit cleric",
    "Jesuit",
]
social = [
    "director of the Peace Corps",  # before arts
    "Peace Corps",  # before politics_govt_law
]
crime = []
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

In [23]:
# Hard_coding info_1 value to capture spiritual for entry
index = df[df["link"] == "https://en.wikipedia.org/wiki/Pasquale_Borgomeo"].index
df.loc[index, "info_1"] = "priest"

# Hard_coding info_2 value to capture business for entry
index = df[df["link"] == "https://en.wikipedia.org/wiki/Jim_Service"].index
df.loc[index, "info_2"] = "business director"  # added to dictionary

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [24]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "sports": sports,
    "arts": arts,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [25]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 54.2 s
Wall time: 54.2 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
48155,19,Dobrica Ćosić,", 92, Serbian writer and politician, President of the Federal Republic of Yugoslavia .",https://en.wikipedia.org/wiki/Dobrica_%C4%86osi%C4%87,23,2014,May,,None,,President of the Federal Republic of,None,None,None,None,None,None,None,None,92.0,None,Serbia,None,1992 1993,3.178054,0,0,0,0,0,1,0,0,1,0,0,0,2
96554,7,Sumy Sadurni,", 32, Chilean photojournalist, traffic collision.",https://en.wikipedia.org/wiki/Sumy_Sadurni,9,2022,March,None,None,,traffic collision,None,None,None,None,None,None,None,None,32.0,None,Chile,None,None,2.302585,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [26]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 9538 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [27]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [28]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [29]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "bounty hunter" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [30]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [31]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "bounty hunter and reality television personality"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [32]:
# Creating lists for each category
politics_govt_law = [
    "deaf rights advocate",
    "government official and gun control advocate",
    "consumer advocate and government official",
    "local government official",
    "and government official",
    "government official",
]

arts = [
    "radio preacher",
    "entertainer and television personality",
    "and reality television personality",
    "radio and television personality",
    "reality television personality",
    "and television personality",
    "television personality",
]
sports = [
    "national hockey team and Pittsburgh Penguins coach",
    "ice hockey trainer and equipment manager",
    "ice hockey Hall of Fame player and coach",
    "Detroit Red Wings hockey player in the s",
    "women baseball and field hockey player",
    "field hockey player and administrator",
    "professional hockey player and coach",
    "ice hockey administrator and referee",
    "ice hockey referee and administrator",
    "professional ice hockey defenseman",
    "Hall of Fame field hockey player",
    "Hall of Fame ice hockey linesman",
    "field hockey player and manager",
    "roller hockey player and coach",
    "field hockey player and coach",
    "ice hockey coach and manager",
    "field hockey representative",
    "professional hockey player",
    "ice hockey administrator",
    "college ice hockey coach",
    "hockey player and coach",
    "NHL ice hockey referee",
    "ice hockey goaltender",
    "ice hockey defenceman",
    "sledge hockey player",
    "field hockey player",
    "ice hockey referee",
    "ice hockey coach",
    "hockey official",
    "hockey referee",
    "hockey player",
    "field hockey",
    "ice hockey",
    "hockey",
    "competitive figure skater as a teenager",
    "figure skater and figure skating coach",
    "pair skater and figure skating",
    "roller derby skater and coach",
    "short track speed skater",
    "long distance ice skater",
    "figure skater and coach",
    "skate and snowboarder",
    "skateboard innovator",
    "Roller derby skater",
    "roller derby skater",
    "figure skater",
    "speed skater",
    "pair skater",
    "skater",
    "weightlifter and fitness centre owner",
    "champion Paralympic weightlifter",
    "world champion weightlifter",
    "heavyweight weightlifter",
    "weightlifter",
    "sailor and nightclub owner",
    "sailor and adventurer",
    "sailboat designer",
    "sailor and coach",
    "land sailor",
    "sailor and",
    "assailant",
    "sailor",
]
sciences = [
    "sailplane designer and pioneer",  # before sports
    "ichthyologist",
    "yacht designer",
    "sceptic",
    "immunologist and eye tissue transplant researcher",
    "gastroenterologist and immunologist",
    "immunologist and cancer researcher",
    "immunologist and AIDS researcher",
    "pathologist and immunologist",
    "virologist and immunologist",
    "cancer immunologist",
    "immunologist",
    "aerodynamics expert at",
    "aeroplane designer",
    "aerospace pioneer",
    "aerodynamicist",
    "aerospace",
    "aero",
]

business_farming = [
    "diamond merchant",
    "wine collector and dealer",
    "wine collector and dealer",  # before academia_humanities
    "pastoral and tourism pioneer",  # before spiritual
    "financier and venture capitalist",
    "venture capitalist and financier",
    "property developer and financier",
    "billionaire financier",
    "corporate  financier",
    "and a financier",
    "financier and",
    "and financier",
    "financier",
    "pawnbroker",
]
academia_humanities = [
    "ichthyologist and musical instrument collector",
    "collector of Harry Houdini memorabilia",
    "diamond merchant and antique collector",
    "optical illusion collector and sceptic",
    "and music collector",
    "toy car collector",
    "antique collector",
    "record collector",
    "book collector",
    "collector",
]
law_enf_military_operator = [
    "sea captain sailor",  # before sports
    "navy sailor",
    "bounty hunter",
]
spiritual = [
    "evangelist and pastor of the Worldwide Church of God",
    "pastor at the University Baptist Church in Waco",
    "Pentecostal evangelical pastor and",
    "Evangelical Lutheran pastor",
    "Independent Baptist pastor",
    "Baptist megachurch pastor",
    "pastor and evangelist",
    "pastor and exorcist",
    "Pentecostal pastor",
    "evangelical pastor",
    "Protestant pastor",
    "megachurch pastor",
    "pastoral theology",
    "reformist pastor",
    "Christian pastor",
    "Baptist pastor",
    "gospel  pastor",
    "senior pastor",
    "pastor and",
    "and pastor",
    "pastor",
]
social = []
crime = ["assailant", "fugitive from justice", "fugitive"]  # before sports
event_record_other = [
    "potato chip collector",  # before academia_humanities
]
other_species = [
    "skateboarding bulldog",  # before sports
]

<IPython.core.display.Javascript object>

In [33]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [34]:
# Hard-coding info_2 value to capture military for entries
index = df[
    df["link"] == "https://en.wikipedia.org/wiki/John_Leake_(NAAFI_manager)"
].index
df.loc[index, "info_2"] = "navy sailor"  # added to dict

index = df[df["link"] == "https://en.wikipedia.org/wiki/Ted_Briggs"].index
df.loc[index, "info_2"] = "navy sailor"

index = df[df["link"] == "https://en.wikipedia.org/wiki/Robert_Walker_(sailor)"].index
df.loc[index, "info_2"] = "navy sailor"

index = df[df["link"] == "https://en.wikipedia.org/wiki/Robert_Stinnett"].index
df.loc[index, "info_2"] = "navy sailor"

index = df[df["link"] == "https://en.wikipedia.org/wiki/Molly_Kool"].index
df.loc[index, "info_2"] = "sea captain sailor"  # added to dict

index = df[df["link"] == "https://en.wikipedia.org/wiki/Susan_Clark_(sailor)"].index
df.loc[index, "info_2"] = "sea captain sailor"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [35]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [36]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 17s
Wall time: 1min 17s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
59678,22,Don McIver,", 80, New Zealand military officer, Chief of General Staff .",https://en.wikipedia.org/wiki/Don_McIver,5,2016,August,", director of the Security Intelligence Service",None,,Chief of General Staff,None,None,None,None,None,None,None,None,80.0,None,New Zealand,None,"1987 1989, director of the New Zealand Security Intelligence Service 1991 1999",1.791759,0,0,0,0,0,0,0,1,0,0,0,0,1
13088,15,Marina Știrbei,", 89, Romanian aviator.",https://en.wikipedia.org/wiki/Marina_%C8%98tirbei,3,2001,July,None,None,,None,None,None,None,None,None,None,None,None,89.0,None,Romania,None,None,1.386294,0,0,0,0,0,0,0,1,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [37]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 9016 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [38]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [39]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [40]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "confectioner" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [41]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [42]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "nun and confectioner"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [43]:
# Creating lists for each category
politics_govt_law = [
    "neo fascist",
    "barrister and",
    "barrister",
    "viceroy",
    "princess and a Muhammad Ali Dynasty member",
    "princess and grandmother of King Felipe VI",
    "princess and granddaughter of Mehmed V",
    "princess of the House of Savoy",
    "princess and expatriate",
    "Sealandic princess",
    "and princess",
    "princess",
    "anti torture advocate",
]

arts = [
    "science fiction and multi winner",
    "science fiction bibliographer",
    "science fiction expert",
    "science fiction and",
    "science fiction fan",
    "science fiction",
    "lyricist known for writing musicals with Adolph Green including",
    "country music performer and member of the Grand Ole Opry",
    "music manager and spokesperson for the Bay City Rollers",
    "electronic music programmer and keyboardist",
    "costume designer of Broadway musicals",
    "classical music radio program host",
    "talent manager and music promoter",
    "founder of exotica musical genre",
    "big band and pop music arranger",
    "music venue owner and promoter",
    "calypsonian and music promoter",
    "bandleader and music arranger",
    "country music and rodeo star",
    '"father of bluegrass" music',
    "classical music impresario",
    "country music entertainer",
    "music promoter and agent",
    "electronic music pioneer",
    "country music performer",
    "percussionist and music",
    "music manager and agent",
    "classical music manager",
    "father of Chicano music",
    "pioneer of Celtic music",
    "and musical instrument",
    "musical administrator",
    "folk music researcher",
    "country music fiddler",
    "music hall performer",
    "music  administrator",
    "folk music promoter",
    "rock music promoter",
    "performer of music",
    "country music star",
    "traditional music",
    "cellist and music",
    "musical arranger",
    "music researcher",
    "musical lyricist",
    "music publicist",
    "music video and",
    "classical music",
    "music education",
    "music promoter",
    "music arranger",
    "music manager",
    "music website",
    "musical agent",
    "music expert",
    "music patron",
    "folk music",
    "and music",
    "musical",
    "music",
    "carnival designer",
    "violist and cellist",
    "classical cellist",
    "and cellist",
    "cellist",
    "Hall of Fame talk radio host",
    "television and radio host",
    "Hall of Fame radio host",
    "talk radio host and",
    "talk radio host",
    "and radio host",
    "radio host and",
    "radio host",
    "ballerina and ballet mistress",
    "ballerina and",
    "ballerina",
    "tenor saxophone player",
    "operatic lyric tenor",
    "operatic tenor",
    "countertenor",
    "opera tenor",
    "lyric tenor",
    "heldentenor",
    "tenor and",
    "tenor",
    "microwave cooking consultant",
    "TV cooking show host",
    "cooking show host",
    "television cook",
    "and cook",
    "cook",
]
sports = [
    "world champion bridge player",
    "professional bridge player",
    "contract bridge player",
    "bridge player",
    "thoroughbred horse breeder",
    "and horse breeder",
    "horse breeder",
    "Paralympic wheelchair curler",
    "world champion curler",
    "Hall of Fame curler",
    "curler",
    "middle distance runner and Commonwealth Games gold medallist",
    "middle distance runner and former world record holder",
    "middle distance runner and",
    "middle and long distance runner",
    "orienteer and mountain runner",
    "marathon and triathlon runner",
    "long distance runner and",
    "middle distance runner",
    "long distance runner",
    "steeplechase runner",
    "orienteering runner",
    "runner and coach",
    "distance runner",
    "sprint runner",
    "fell runner",
    "runner",
    "judoka",
    "race car driver and member of the NASCAR Hall of Fame",
    "NASCAR driver and ARCA race car driver owner",
    "Formula One and Grand Prix race car driver",
    "race car driver and hot rod builder",
    "race car driver and team owner",
    "Hall of Fame race car driver",
    "race car driver and mechanic",
    "Formula One race car driver",
    "race car driver and owner",
    "NASCAR race car driver",
    "race car driver and",
    "race car driver",
    "Baseball player who was the first to come out as gay",
    "Baseball player for the Philadelphia Athletics",
    "Baseball player and manager",
    "Baseball player and coach",
    "former Baseball player",
    "Baseball player",
    "Hall of Fame gymnast",
    "gymnastics coach",
    "rhythmic gymnast",
    "gymnast",
    "racecar driver and member of the NASCAR Hall of Fame",
    "professional racecar driver",
    "and racecar driver",
    "racecar driver",
    "baseball umpire and supervisor",
    "Hall of Fame baseball umpire",
    "baseball umpire",
    "Hall of Fame badminton player",
    "badminton player and coach",
    "badminton player and",
    "badminton player",
    "Test cricket umpire",
    "test cricket umpire",
    "cricket umpire",
]
sciences = [
    "paleontologist who revolutionized understanding of dinosaurs",
    "entomologist and paleontologist",
    "malacologist and paleontologist",
    "vertebrate paleontologist",
    "paleontologist",
    "arachnologist and myriapodologist",
    "entomologist and arachnologist",
    "arachnologist and",
    "arachnologist",
    "cardiologist who invented the technique of coronary bypass surgery",
    "cardiologist and expert on hypertension",
    "paediatric cardiologist",
    "pediatric cardiologist",
    "cardiologist",
    "child psychoanalyst",
    "psychoanalyst",
    "gynaecologist who is among the oldest men to have fathered a child",
    "gynaecologist and reproductive medicine researcher",
    "obstetrician and gynaecologist",
    "gynecologist and obstetrician",
    "obstetrician and gynecologist",
    "entomologist and ecologist",
    "statistical ecologist",
    "landscape ecologist",
    "and media ecologist",
    "plant ecologist",
    "paleoecologist",
    "deep ecologist",
    "gynaecologist",
    "geo ecologist",
    "gynecologist",
    "ecologist",
    "virologist credited with eradicating polio in",
    "epidemiologist and virologist",
    "virologist and paediatrician",
    "plant virologist",
    "virologist",
    "criminologist",
    "pharmacologist and biodynamic agriculturalist",
    "Nobel Prize winning pharmacologist",
    "physiologist and pharmacologist",
    "pharmacologist and physiologist",
    "behavioral pharmacologist",
    "clinical pharmacologist",
    "psychopharmacologist",
    "neuropharmacologist",
    "pharmacologist",
    "pathologist who specialized in sickle cell anemia and hematology",
    "pathologist and cancer researcher",
    "neurologist and neuropathologist",
    "pathologist and toxicologist",
    "veterinary pathologist",
    "paediatric pathologist",
    "forensic pathologist",
    "chemical pathologist",
    "cancer pathologist",
    "dermatopathologist",
    "animal pathologist",
    "plant pathologist",
    "phytopathologist",
    "oral pathologist",
    "neuropathologist",
    "pathologist and",
    "pathologist",
    "sexologist and psychotherapist",
    "child psychotherapist",
    "psychotherapist",
]

business_farming = ["founder of PowerBar", "winemaker and", "winemaker", "confectioner"]
academia_humanities = [
    "library",
    "emeritus curator at the Smithsonian Institution National Museum of Natural History",
    "archeologist and former curator at the Smithsonian Institution",
    'museum curator and one of the "Monuments Men"',
    "manuscripts curator at the Museum",
    "Smithsonian Institution curator",
    "egyptologist and curator",
    "educationist and curator",
    "curator of contemporary",
    "curator of paintings",
    "photography curator",
    "and museum curator",
    "museum curator",
    "and curator",
    "curator",
]
law_enf_military_operator = [
    "astronaut in Mercury",
    "candidate astronaut",
    "former astronaut",
    "NASA astronaut",
    "astronaut",
    "resistant",
    "air marshal and Director General of Intelligence",
    "air marshal and George Cross recipient",
    "Air Force air marshal",
    "air marshall",
    "air marshal",
]
spiritual = [
    "Christianity preacher and gospel",
    "prelate from  Catholic Association",
    "prelate of the Catholic Church",
    "prelate in the Catholic Church",
    "catholic prelate and cardinal",
    "Catholic clandestine prelate",
    "Catholic laicized prelate",
    "Catholic Cardinal prelate",
    "Orthodox Old Rite prelate",
    "Eastern Orthodox prelate",
    "United Methodist prelate",
    "Church of South prelate",
    "Episcopalian prelate",
    "Catholic ex prelate",
    "Episcopal prelate",
    "Apostolic prelate",
    "Mar Thoma prelate",
    "episcopal prelate",
    "Church of prelate",
    "Angelican prelate",
    "Orthodox prelate",
    "Lutheran prelate",
    "Maronite prelate",
    "Coptic prelate",
    "Jewish prelate",
    "Mormon prelate",
    "prelate",
    "biblical",
    "Catholic nun and",
    "Apostolic nuncio",
    "Benedictine nun",
    "Poor Clare nun",
    "Catholic nun",
    "and nun",
    "nun",
]
social = [
    "and confectioner",  # before business_farming
]
crime = [
    "terrorist involved in the Glasgow International Airport attack",
    "founder and commander in chief of terrorist organization FARC",
    "terrorist and a commander of Abu Sayyaf",
    "islamist terrorist group leader",
    "Arabian suspected terrorist",
    "terrorist in Bali bombings",
    "al Qaeda terrorist",
    "domestic terrorist",
    "Arabian terrorist",
    "and terrorist",
    "terrorist",
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [44]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [45]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 2min 38s
Wall time: 2min 38s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
71568,6,Coşkun Büktel,", 67, Turkish playwright.",https://en.wikipedia.org/wiki/Co%C5%9Fkun_B%C3%BCktel,5,2018,November,None,None,,None,None,None,None,None,None,None,None,None,67.0,None,Turkey,None,None,1.791759,0,0,0,0,0,1,0,0,0,0,0,0,1
44679,2,Terry Clawson,", 73, English rugby league player.",https://en.wikipedia.org/wiki/Terry_Clawson,14,2013,September,None,None,,None,None,None,None,None,None,None,None,None,73.0,None,United Kingdom of Great Britain and Northern Ireland,None,None,2.708050,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [46]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 7672 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [47]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [48]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [49]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "President of" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [50]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [51]:
# Example code to quick-check a specific entry
df[df["info_2"] == "puppeteer and visual effects technician ; multiple sclerosis"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
54136,18,Brock Winkless,", 55, American puppeteer and visual effects technician ; multiple sclerosis.",https://en.wikipedia.org/wiki/Brock_Winkless,3,2015,July,", ,",None,puppeteer and visual effects technician ; multiple sclerosis,None,None,None,None,None,None,None,None,None,55.0,None,United States of America,None,", ,",1.386294,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [52]:
# Creating lists for each category
politics_govt_law = [
    "advocate for women education",  # before academia_humanities
    "sexual education advocate",
    "radical",
    "Conservative Member of Parliament and former government minister",  # before spiritual
    "president of land and former prime minister of the Republic",
    "minister for social welfare in the Punjab province",
    "opposition leader and former prime minister",
    "minister and Lord of Appeal in Ordinary",
    "minister of labor and social security",
    "Haryana four time chief minister",
    "Conservative government minister",
    "first chief minister of Sikkim",
    "deputy and former minister",
    "chief minister of the ACT",
    "Liberal cabinet minister",
    "and government minister",
    "finance minister since",
    "Nonconformist minister",
    "Prime minister of land",
    "deputy prime minister",
    "n minister of Housing",
    "and deputy minister",
    "government minister",
    "interior minister",
    "highways minister",
    "foreign minister",
    "Cabinet minister",
    "cabinet minister",
    "finance minister",
    "Foreign minister",
    "prime minister",
    "advocate for disadvantaged students",
    "child safety campaigner",
    "WHO official",
    "prince and Head of the House of Hesse",
    "prince and head of the dynasty",
    "Qing dynasty imperial prince",
    "prince and Archduke of Este",
    "Imperial prince",
    "Arabian prince",
    "princeling",
    "prince",
]

arts = [
    "who created the Ladybird books",
    "Hall of Fame lyricist",
    "Broadway lyricist",
    "and lyricist",
    "lyricist",
    "theatrical and television set and costume designer",
    "costume designer and Academy Award winner",
    "Tony Award winning costume designer",
    "and Broadway costume designer",
    "stage and costume designer",
    "set and costume designer",
    "costume designer",
    "keyboardist and a founding member of the progressive rock group Camel",
    "keyboardist for Bruce Springsteen & The E Street Band",
    "keyboardist for Janis Joplin and The Band",
    "keyboardist and sound technician",
    "progressive rock keyboardist",
    "keyboardist and vocalist",
    "Hall of Fame keyboardist",
    "keyboardist of Spirit",
    "blues keyboardist",
    "rock keyboardist",
    "funk keyboardist",
    "keyboard player",
    "keyboardist",
    "diarist",
    "role playing game designer",
    "roleplaying game designer",
    "board game designer",
    "game designer and",
    "wargame designer",
    "game designer",
    "gardening expert and radio presenter",
    "voice over and radio presenter",
    "TV and radio presenter",
    "DJ and radio presenter",
    "BBC radio presenter",
    "radio presenter and",
    "radio presenter",
    "puppeteer and visual effects technician ; multiple sclerosis",
    "ventriloquist and puppeteer",
    "puppeteer and ventriloquist",
    "wayang golek puppeteer",
    "wayang puppeteer",
    "potehi puppeteer",
    "puppeteer",
    "ceramicist and potter",
    "Cochiti Pueblo potter",
    "potter and ceramist",
    "studio potter",
    "potter and",
    "and potter",
    "potter",
    "multi instrumentalist and entertainer",
    "TV entertainer most active in y",
    "talk show host and entertainer",
    "guitar maker and entertainer",
    "trumpeter and entertainer",
    "transvestite entertainer",
    "transsexual entertainer",
    "entertainer and manager",
    "vaudeville entertainer",
    "children entertainer",
    "street entertainer",
    "comic entertainer",
    "drag entertainer",
    "entertainer and",
    "entertainer",
]
sports = [
    "motorcycle speedway rider and Formula One driver",
    "motorcycle speedway rider and coach",
    "speedway and ice speedway rider",
    "speedway rider and promoter",
    "motorcycle speedway rider",
    "speedway rider",
    "handball player and coach",
    "East handball player",
    "handball player",
    "Hall of Fame college baseball coach",
    "college baseball coach and player",
    "basketball and baseball coach",
    "Hall of Fame baseball coach",
    "baseball coach and official",
    "baseball coach and manager",
    "college baseball coach",
    "baseball coach",
]
sciences = [
    "daffodil breeder",
    "NASA space science administrator and a leader in satellite communications",
    "computer science pioneer",
    "materials science expert",
    "mathematics and science",
    "neuroscience researcher",
    "and information science",
    "science and technology",
    "paediatric sciences",
    "cognitive science",
    "chemical sciences",
    "computer science",
    "neuroscience",
    "science",
    "clinical and research pediatrician",
    "pediatrician and founding",
    "pediatrician and",
    "pediatrician",
    "paleoentomologist and coleopterist",
    "entomologist and lepidopterist",
    "entomologist and toxicologist",
    "forensic entomologist",
    "entomologist and",
    "entomologist",
    "agronomist and tea expert",
    "agronomist",
    "neurologist and epileptologist",
    "pediatric neurologist",
    "neurologist and",
    "neurologist",
    "paediatrician and sudden infant death syndrome researcher",
    "paediatrician and",
    "paediatrician",
    "epidemiologist and infectionist",
    "epidemiologist and oncologist",
    "cardiovascular epidemiologist",
    "dental epidemiologist",
    "epidemiologist and",
    "epidemiologist",
    "cosmonaut trainer",
    "first cosmonaut",
    "cosmonaut",
]


business_farming = [
    "wine pioneer and vineyard owner",
    "wine pioneer",
]
academia_humanities = [
    "education reformer and administrator",
    "educationist and intellectual",
    "educational administrator",
    "adult education innovator",
    "physical education expert",
    "education administrator",
    "educational researcher",
    "educational consultant",
    "educational theorist",
    "education consultant",
    "education proponent",
    "educationalist and",
    "education reformer",
    "education official",
    "educational leader",
    "educationalist and",
    "education leader",
    "and educationist",
    "educationalist",
    "educationist",
    "educational",
    "education",
    "classicist and digital humanist",
    "secular humanist",
    "humanist",
]
law_enf_military_operator = [
    "spy and the first chief of intelligence agency",
    "MI agent and spy for the Union",
    "who was a spy for the Union",
    "atomic spy for the Union",
    "spy and double agent",
    "insurgent spy and",
    "spy for the Union",
    "spy for the Stasi",
    "colonel and spy",
    "former KGB spy",
    "Union spy in",
    "alleged spy",
    "Allied spy",
    "spy chief",
    "and spy",
    "spy",
    "deputy minister of intelligence",  # before spiritual
    "railroad",
]
spiritual = [
    "Lingayat spiritual leader",
    "minister of the Church of the Intercession in Harlem",
    "evangelist and Southern Baptist minister",
    "Unitarian Universalist minister",
    "ist and Presbyterian minister",
    "Romani Pentecostal minister",
    "Southern Baptist minister",
    "Protestant minister and",
    "Congregational minister",
    "Christian minister and",
    "Unitarian minister and",
    "Presbyterian minister",
    "Pentecostal minister",
    "evangelical minister",
    "Protestant minister",
    "Methodist minister",
    "Christian minister",
    "Church of minister",
    "religious minister",
    "minister in Harlem",
    "Lutheran minister",
    "Nazarene minister",
    "ordained minister",
    "Baptist minister",
    "baptist minister",
    "Quaker minister",
    "youth minister",
    "minister and",
    "minister",
    "Pentecostal clergyman",
    "Church of clergyman",
    "Episcopal clergyman",
    "Anglican clergyman",
    "Catholic clergyman",
    "Lutheran clergyman",
    "Maronite clergyman",
    "Orthodox clergyman",
    "Anglican clergy",
    "Jewish  clergy",
    "clergyman and",
    "clergyman in",
    "clergyman",
]
social = [
    "community organizer",
]
crime = []
event_record_other = [
    "Hispanic John Jay College of Criminal Justice student",
    "student at South Hadley High School",
    "student and hazing victim",
    "high school student",
    "centenarian student",
    "student and victim",
    "graduate student",
    "exchange student",
    "college student",
    "honors student",
    "Ph D student",
    "law student",
    "student",
]
other_species = []

<IPython.core.display.Javascript object>

In [53]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

In [54]:
# Hard-coding info_2 value for entry to capture event_record_other category only
index = df[df["link"] == "https://en.wikipedia.org/wiki/Eve_Carson"].index
df.loc[index, "info_2"] = "student"

# Hard-coding cause_of_death for entry with value in info_2
index = df[df["link"] == "https://en.wikipedia.org/wiki/Brock_Winkless"].index
df.loc[index, "cause_of_death"] = "multiple sclerosis"

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [55]:
# Combining separate lists into one dictionary
known_for_dict = {
    "politics_govt_law": politics_govt_law,
    "social": social,
    "business_farming": business_farming,
    "sciences": sciences,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
    "academia_humanities": academia_humanities,
    "spiritual": spiritual,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [56]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 2min 13s
Wall time: 2min 13s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
90485,4,John M. Patterson,", 99, American politician, attorney general of Alabama.",https://en.wikipedia.org/wiki/John_M._Patterson,49,2021,June,and governor,None,,attorney general of Alabama,None,None,None,None,None,None,None,None,99.0,None,United States of America,None,1955 1959 and governor 1959 1963,3.912023,0,0,0,0,0,0,0,0,1,0,0,0,1
55846,1,Jim Loscutoff,", 85, American basketball player , complications of pneumonia and Parkinson's disease.",https://en.wikipedia.org/wiki/Jim_Loscutoff,13,2015,December,"Boston Celtics, NBA champion ,",None,,complications of pneumonia and Parkinson disease,None,None,None,None,None,None,None,None,85.0,None,United States of America,None,"Boston Celtics, NBA champion 1957, 1959 1964",2.639057,0,0,0,0,0,0,1,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [57]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 6840 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [58]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [59]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [60]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "UFO" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [61]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [62]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "al theorist"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [63]:
# Creating lists for each category
politics_govt_law = [
    "two term member of County Board of Supervisors",
    "polemicist and conspiracy theorist",
    "RMS Titanic conspiracy theorist",
    "civil rights movement leader",
    "and conspiracy theorist",
    "civil rights campaigner",
    "civil rights pioneer",
    "conspiracy theorist",
    "civil rights worker",
    "law and economics",
    "health economics",
    "economic advisor",
    "Western Marxist",
    "and economic",
    "civil rights",
    "economics",
    "economic",
    "planning",
    "Marxist",
]

arts = [
    "children television host and personality",
    "interior designer and television host",
    "television reporter and interviewer",
    "television and radio talk show host",
    "Bravo television network trainer on",
    "radio and television broadcasting",
    "television  for over four decades",
    "ventriloquist and television and",
    "television host and interviewer",
    "talk radio and television host",
    "state television correspondent",
    "television and  and presenter",
    "television news correspondent",
    "CBC radio and television host",
    "satirist and television host",
    "television reporter and host",
    "television fitness presenter",
    "reporter and television host",
    "television weather presenter",
    "pitchman and television host",
    "television production mogul",
    "public television innovator",
    "television camera operator",
    "television news presenter",
    "radio and television host",
    "television quiz show host",
    "children television host",
    "television color analyst",
    "television station owner",
    "television correspondent",
    "television news reporter",
    "television reality star",
    "television reporter and",
    "reality television star",
    "public television host",
    "television host and DJ",
    "television  presenter",
    "television newsreader",
    "television show host",
    "and television host",
    "television reporter",
    "television designer",
    "television station",
    "television pioneer",
    "television hostess",
    "television newsman",
    "television anchor",
    "television  since",
    "television emcee",
    "television host",
    "television star",
    "television and",
    "urban planning",  # before politics_govt_law
    "television",
]
sports = []
sciences = [
    "founder and principal theorist of Re evaluation Counseling",  # before academia_humanities
    "pioneer in anesthesiology and pain management",  # before business_farming
    "information security specialist",
    "of communication and media",
    "family planning pioneer",  # before politics_govt_law
    "information researcher",
    "who built Alaska first",
    "nursing administrator",
    "telecommunications",
    "nursing assistant",
    "control theorist",
    "early UFOlogist",
    "analytic number",
    "graph theorist",
    "UFO researcher",
    "communications",
    "communication",
    "consciousness",
    "information",
    "nursing",
    "number",
    "UFO",
]


business_farming = [
    "product planning manager",  # before politics_govt_law
    "management consultant",
    "management studies",
    "marketing manager",
    "marketing agent",
    "management guru",
    "organizational",
    "organisational",
    "management",
    "marketing",
]

academia_humanities = [
    "theorist in Arte Povera movement",
    "theorist and archivist",
    "pedagogical theorist",
    "curriculum theorist",
    "literary theorist",
    "cultural theorist",
    "lesbian theorist",
    "design theorist",
    "social theorist",
    "al theorist",
    "theorist",
]
law_enf_military_operator = []
spiritual = [
    "Yup'ik traditional healer",
    "Shia cleric and ayatollah",
    "traditional healer",
    "Shia cleric",
    "Shia",
]
social = []
crime = []
event_record_other = [
    "alleged UFO witness",  # before sciences
]
other_species = []

<IPython.core.display.Javascript object>

In [64]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(spiritual)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [65]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "event_record_other": event_record_other,
    "sciences": sciences,
    "arts": arts,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "other_species": other_species,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [66]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 1min 4s
Wall time: 1min 4s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
52313,7,Trevor Griffin,", 74, Australian politician, Attorney-General of South Australia .",https://en.wikipedia.org/wiki/Trevor_Griffin,3,2015,March,",",None,,Attorney General of South,None,None,None,None,None,None,None,None,74.0,None,Australia,None,"1979 1982, 1993 2001",1.386294,0,0,0,0,0,0,0,0,1,0,0,0,1
25576,11,John Rutsey,", 55, Canadian drummer , heart attack.",https://en.wikipedia.org/wiki/John_Rutsey,14,2008,May,Rush,None,,heart attack,None,None,None,None,None,None,None,None,55.0,None,Canada,None,Rush,2.708050,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [67]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 6681 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [68]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [69]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [70]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "driver" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [71]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [72]:
# # Example code to quick-check a trspecific entry
# df[df["info_2"] == "driver"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [73]:
# Creating lists for each category
politics_govt_law = [
    "assisted suicide advocate",
    "proponent of the single market and Vice President of the Commission",
    "nd President of the Republic and Emperor of Central",
    "first Vice President of the Navajo Nation",
    "President of the Maryland State Senate",
    "President of Federal Reserve Bank of",
    "revolutionary and President of istan",
    "President of the People Republic of",
    "President of the Arab Republic",
    "Vice President of the Republic",
    "President of during The Junta",
    "President of the Bundestag",
    "first President of Senegal",
    "statesman and President of",
    "President of the Republic",
    "former President of South",
    "President of from to",
    "th President of the",
    "Sixth President of",
    "President of the",
    "th President of",
    "nd President of",
    "President of",
    "white supremacist and Holocaust denier",  # before event_record_other
    "solicitor and Holocaust denier"
    "noblewoman and member of the House of Bourbon Two Sicilies",
    "noblewoman and daughter of Victor Emmanuel III of",
    "noblewoman and daughter of King Alfonso XIII",
    "noble and member of the Vanderbilt family",
    "noble and fourth wife of emperor Puyi",
    "noblewoman and monarchist",
    "noble and Infante",
    "noblewoman and",
    "and noblewoman",
    "noblewoman",
    "nobleman",
    "noble",
    "congressman and labor leader",
    "labor leader",
    "bus driver",  # before event_record_other
]

arts = [
    "fashion and bridal wear designer and retailer",
    "designer and the founder of Landor Associates",
    "stone letter carver and typeface designer",
    "designer for Dansk International Designs",
    "production designer and set decorator",
    "Hall of Fame theatrical set designer",
    "designer of Raleigh Chopper bicycle",
    "electric guitar designer and maker",
    "interior decorator and designer",
    "furniture and interior designer",
    "typographer and book designer",
    "interior and product designer",
    "designer of printed textiles",
    "theatrical lighting designer",
    "designer and window dresser",
    "costume and makeup designer",
    "designer of women lingerie",
    "modernist textile designer",
    "ceramic and glass designer",
    "rock album cover designer",
    "Emmy winning set designer",
    "fashion muse and designer",
    "designer of the star flag",
    "costume jewelry designer",
    "and production designer",
    "designer of board games",
    "roller coaster designer",
    "decorator and designer",
    "production designer",
    "embroidery designer",
    "theatrical designer",
    "restaurant designer",
    "scenic designer and",
    "women shoe designer",
    "furniture designer",
    "golf club designer",
    "landscape designer",
    "animation designer",
    "newspaper designer",
    "interior designer",
    "lighting designer",
    "typeface designer",
    "footwear designer",
    "ceramics designer",
    "logotype designer",
    "textile designer",
    "jewelry designer",
    "handbag designer",
    "scenic designer",
    "grotto designer",
    "puzzle designer",
    "puppet designer",
    "stage designer",
    "sound designer",
    "urban designer",
    "glass designer",
    "chief designer",
    "shoe designer",
    "type designer",
    "book designer",
    "flag designer",
    "set designer",
    "web designer",
    "toy designer",
    "TV designer",
    "designer",
    "board shaper",
    "literary guardian and the only child of Agatha Christie",
    "literary executor of James Joyce",
    "talent and literary agent",
    "literary preservationist",
    "literary figure",
    "literary agent",
    "literary",
    "toymaker",
    "news correspondent and essayist",
    "and essayist",
    "essayist",
]
sports = [
    "World surfing champion",
    "professional surfer",
    "surfing legend",
    "windsurfer",
    "surfing",
    "surfer",
    "surf",
    "rodeo cowboy and professional poker player",
    "Hall of Fame poker player and",
    "backgammon and poker player",
    "bookmaker and poker player",
    "professional poker player",
    "Hall of Fame poker player",
    "bridge and poker player",
    "professional poker",
    "poker player and",
    "poker player",
    "former President of FISA and later FIA",  # before politics_govt_law
    "baseball third baseman outfielder who played for the Cincinnati Reds",
    "baseball manager and member of the Baseball Hall of Fame",
    "first Black baseball pitcher to win a World Series game",
    "baseball pitcher and member of the MLB Hall of Fame",
    "baseball relief pitcher for the St Louis Cardinals",
    "baseball first base coach for the Tulsa Drillers",
    "controversial son of baseball great Ted Williams",
    "longtime minor league baseball record holder",
    "baseball second basemen and shortstop",
    "baseball pitcher and pitching coach",
    "baseball pitcher for Boston Red Sox",
    "baseball infielder and outfielder",
    "baseball outfielder and manager",
    "Hall of Fame baseball manager",
    "baseball and softball player",
    "college baseball head coach",
    "baseball manager and coach",
    "baseball clubhouse manager",
    "AAGPBL baseball pitcher",
    "baseball pitcher player",
    "baseball second baseman",
    "baseball college coach",
    "baseball right fielder",
    "baseball third baseman",
    "baseball administrator",
    "baseball  researcher",
    "baseball researcher",
    "baseball club owner",
    "baseball outfielder",
    "baseball shortstop",
    "baseball promoter",
    "baseball pitcher",
    "baseball manager",
    "baseball catcher",
    "baseball figure",
    "baseball batboy",
    "baseball owner",
    "baseball fan",
    "baseball",
    "stock car race driver and member of the NASCAR Hall of Fame",  # before event_record_other
    "rally driver and principal of the Toyota F racing team",
    "NASCAR driver and member of the NASCAR Hall of Fame",
    "chief test driver for Toyota Motor Company",
    "Hall of Fame NASCAR driver and owner",
    "retired stock car and NASCAR driver",
    "car racing team owner and driver",
    "USAC champion midget car driver",
    "sulky driver and horse trainer",
    "Alfa Romeo works' test driver",
    "motor racing and rally driver",
    "NASCAR driver and team owner",
    "professional drifting driver",
    "rally driver and team owner",
    "off road race truck driver",
    "Hall of Fame NASCAR driver",
    "NASCAR Busch Series driver",
    "world tour rally driver",
    "NASCAR driver and owner",
    "and Formula Two driver",
    "racing and test driver",
    "champion NASCAR driver",
    "IndyCar Series driver",
    "former NASCAR driver",
    "Formula One driver",
    "NASCAR race driver",
    "racing car driver",
    "V Supercar driver",
    "Grand Prix driver",
    "rally raid driver",
    "stock car driver",
    "rally co driver",
    "IndyCar driver",
    "NASCAR driver",
    "rally driver",
    "race driver",
    "test driver",
    "NHRA driver",
]
sciences = [
    "designer of the Pitts Special and other aircraft",  # before arts
    "aircraft designer and rocketry pioneer",
    "automobile designer and constructor",
    "designer of the Uzi submachine gun",
    "designer of hi fi audio equipment",
    "racing car and engine designer",
    "racecar and aircraft designer",
    "engine designer and builder",
    "integrated circuit designer",
    "marine equipment designer",
    "boat builder and designer",
    "mechanic and car designer",
    "boat designer and builder",
    "and automobile designer",
    "and aircraft designer",
    "synthesizer designer",
    "and weapons designer",
    "automobile designer",
    "automotive designer",
    "motorcycle designer",
    "custom car designer",
    "spacecraft designer",
    "and rocket designer",
    "motorboat designer",
    "powerboat designer",
    "aircraft designer",
    "computer designer",
    "firearms designer",
    "and boat designer",
    "product designer",
    "weapons designer",
    "car designer and",
    "rocket designer",
    "glider designer",
    "automotive and",
    "boat designer",
    "auto designer",
    "tank designer",
    "car designer",
    "gun designer",
    "pinball",
    "physiologist and sleep researcher",
    "physiologist and nutritionist",
    "nutritional physiologist",
    "plant physiologist",
    "fetal physiologist",
    "psychophysiologist",
    "neurophysiologist",
    "physiologist",
    "pharmacist and scientific researcher",
    "pharmacist and vilification victim",
    "pharmacist and",
    "pharmacist",
]

business_farming = [
    "billionaire automotive manufacturer",
    "automotive products manufacturer",
    "billionaire drug manufacturer",
    "knife maker and manufacturer",
    "roller coaster manufacturer",
    "wind turbine manufacturer",
    "generic drug manufacturer",
    "bicycle tool manufacturer",
    "racing car manufacturer",
    "sunscreen manufacturer",
    "clothing manufacturer",
    "chemical manufacturer",
    "footwear manufacturer",
    "bushwear manufacturer",
    "textile manufacturer",
    "cigar manufacturer",
    "food manufacturer",
    "oven manufacturer",
    "toy manufacturer",
    "manufacturer",
    "pioneer in the field of industrial",
    "industrial relations",
    "industrial manager",
    "industrial",
    "carpet distributor",
    "nightclub owner and property developer",
    "billionaire property developer",
    "property developer and",
    "property developer",
]
academia_humanities = [
    "former President of Hillsdale College",  # before politics_govt_law
    "President of Gadjah Mada University",
    "President of Princeton University",
    "President of University of Beirut",
    "President of Hebrew Union College",
    "lector",
    "archivist who led the National Archives and Records Administration",
    "archivist for the Rudolf Nureyev Foundation",
    "archivist and administrator",
    "archivist",
]
law_enf_military_operator = [
    "anti communist resistance fighter",
    "Jewish resistance fighter",
    "WWII resistance fighter",
    "resistance fighter",
    "taxi driver",  # before event_record_other
]
spiritual = [
    "President of The Church of Jesus Christ of Latter day Saints",  # before politics_govt_law
    "President of the Church of",
    "founder and leader of religious movement The Family International",
    "religious sect founder and former",
    "Catholic religious brother",
    "Catholic religious sister",
    "religious cult leader",
    "religious sect leader",
    "religious extremist",
    "religious studies",
    "religious advisor",
    "religious figure",
    "religious mystic",
    "religious sister",
    "religious books",
    "religious chief",
    "religious",
    "evangelist and alleged doomsday predictor",
    "wife of evangelist Billy Graham",
    "fundamentalist evangelist and",
    "gospel  Christian evangelist",
    "Pentecostal evangelist",
    "Christian evangelist",
    "Lutheran evangelist",
    "evangelist",
]
social = [
    "President of Orbis International",  # before politics_govt_law
]
crime = [
    "and Holocaust perpetrator",  # before event_record_other
]
event_record_other = ["and Holocaust", "vilification victim", "driver"]
other_species = []

<IPython.core.display.Javascript object>

In [74]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [75]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "sports": sports,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "other_species": other_species,
    "arts": arts,
    "event_record_other": event_record_other,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [76]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 2min 49s
Wall time: 2min 49s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
78225,18,Marron Curtis Fort,", 81, American-born German linguist.",https://en.wikipedia.org/wiki/Marron_Curtis_Fort,7,2019,December,None,None,,None,None,None,None,None,None,None,None,None,81.0,None,United States of America,Germany,None,2.079442,0,0,0,1,0,0,0,0,0,0,0,0,1
65837,25,Pinito del Oro,", 86, Spanish trapeze artist .",https://en.wikipedia.org/wiki/Pinito_del_Oro,12,2017,October,Ringling Bros and Barnum & Bailey Circus,None,,None,None,None,None,None,None,None,None,None,86.0,None,Spain,None,Ringling Bros and Barnum & Bailey Circus,2.564949,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [77]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 5932 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [337]:
# # Obtaining values for column and their counts
# roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [336]:
# # Code to check each value
# roles_list.pop()

<IPython.core.display.Javascript object>

In [335]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "Orthodox Jewish" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

<IPython.core.display.Javascript object>

In [334]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

<IPython.core.display.Javascript object>

In [333]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "Orthodox Jewish intellectual and polymath"]

<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [332]:
# Creating lists for each category
politics_govt_law = [
    "Christian boy who became a symbol",  # before spiritual
    "widow of Chiang Ching kuo and First Lady of the Republic of on",
    "former First Lady of Montana",
    "First Lady of New Hampshire",
    "First Lady of West Virginia",
    "First Lady of North Dakota",
    "gubernatorial First Lady",
    "First Lady of Maryland",
    "First Lady of Illinois",
    "First Lady of Michigan",
    "First Lady of Alabama",
    "First Lady of the",
    "South First Lady",
    "First Lady and",
    "First Lady of",
    "First Lady",
    "and animal welfare advocate",
    "Deputy Prime Minister and Mayor of Bucharest during the Communist era",
    "widow of former Prime Minister and President Georges Pompidou",
    "Prime Minister of the and first female speaker of the House",
    "Prime Minister of the Socialist Republic of",
    "Prime Minister of under Mobutu Sese Seko",
    "wife of Prime Minister Morgan Tsvangirai",
    "Prime Minister and Member of Lok Sabha",
    "former President and Prime Minister of",
    "wife of Prime Minister Yitzhak Rabin",
    "former Prime Minister of the Union",
    "Prime Minister of the Republic",
    "former Deputy Prime Minister",
    "Prime Minister of Southern",
    "first Prime Minister of",
    "First Prime Minister of",
    "Prime Minister of the",
    "former Prime Minister",
    "th Prime Minister of",
    "Prime Minister since",
    "nd Prime Minister of",
    "st Prime Minister of",
    "Prime Minister wife",
    "Prime Minister of",
    "Prime Minister",
    "iconoclast",
    "public official",
    "labour union leader",
    "union leader and",
    "union leader",
    "Lieutenant Governor of Arkansas since",
    "Communist revolutionary figure",
    "communist and revolutionary",
    "communist revolutionary",
    "Communist revolutionary",
    "marxist revolutionary",
    "revolutionary and" "revolutionary",
    "statesman and founding father of the Republic",
    "General of Internal Service and statesman",
    "and statesman",
    "statesman",
    "chairman of the Jewish Defence League",  # before spiritual
    "Jewish leader",
]

arts = [
    "Tony Award winning publicist",
    "publicist and talent manager",
    "publicist",
    "percussionist and band leader",
    "percussionist and bandleader",
    "salsa percussionist",
    "percussionist",
    "trumpeter and flugelhorn player",
    "trumpeter and bandleader",
    "bandleader and trumpeter",
    "trumpeter and vocalist",
    "trumpeter and arranger",
    "classical trumpeter",
    "highlife trumpeter",
    "trumpet player",
    "trumpeter",
    "clarinetist and bandleader",
    "big band clarinetist",
    "clarinetist",
    "mezzo",
    "drum and bass DJ and co founder of record label Metalheadz",
    "DJ and founder of The Loft",
    "deep house and trance DJ",
    "radio DJ and presenter",
    "radio DJ and TV host",
    "R&B and rock DJ",
    "hip hop DJ",
    "reggae DJ",
    "radio DJ",
    "BBC DJ",
    "DJ",
    "founder of DX Ball",
    "radio programmer",  # before sciences
    "newspaper reporter and Pulitzer Prize winner",
    "Pulitzer Prize winning newspaper reporter",
    "BBC reporter and radio innovator",
    "Pulitzer Prize winning reporter",
    "reporter and war correspondent",
    "BBC reporter for Radio Kent",
    "broadcast news reporter",
    "investigative reporter",
    "anchor and reporter",
    "Indymedia reporter",
    "freelance reporter",
    "newspaper reporter",
    "crime reporter",
    "news reporter",
    "and reporter",
    "ITN reporter",
    "reporter",
    "humorist Georges Bernier",
    "humorist and",
    "and humorist",
    "humorist",
    'known as "the Cosmic Muffin"',
    "operatic bass baritone at the Vienna State Opera and Metropolitan Opera",
    "bass baritone at the Metropolitan Opera",
    "baritone with the Metropolitan Opera",
    "cantor and operatic baritone",
    "operatic bass baritone",
    "operatic baritone",
    "opera baritone",
    "bass baritone",
    "baritone",
    "tightrope walker",
]
sports = [
    "mountain climber and adventurer",
    "rock climber and adventurer",
    "kayaker and adventurer",
    "ocean kayak adventurer",
    "and adventurer",
    "adventurer",
    "Hall of Fame water polo player",
    "water polo player and coach",
    "polo player and coach",
    "water polo player",
    "polo player",
    "Thoroughbred horse trainer and owner",
    "thoroughbred horse trainer",
    "horse trainer and gambler",
    "champion horse trainer",
    "race horse trainer",
    "horse trainer",
    "international volleyball player",
    "volleyball player and coach",
    "and volleyball player",
    "era volleyball player",
    "volleyball player",
    "volleyball coach",
    "cricket and netball international",
    "cricket administrator and manager",
    "cricket and horse racing",
    "South cricket official",
    "cricket administrator",
    "cricket team captain",
    "cricket official",
    "cricket chairman",
    "cricket coach",
    "cricket",
    "high jumper and world record holder",
    "ski jumper who competed for USSR",
    "triple jumper and long jumper",
    "and base jumper",
    "triple jumper",
    "high jumper",
    "long jumper",
    "BASE jumper",
    "ski jumper",
    "showjumper",
]
sciences = [
    "research mycologist",
    "mycologist",
    "paleoceanographer",
    "oceanographer",
    "gemologist",
    "computer programmer known for GNU Debugger",
    "cryptographer and programmer",
    "Computer programmer and",
    "video game programmer",
    "computer programmer",
    "software programmer",
    "game programmer",
    "programmer",
    "medical oncologist and researcher",
    "biotechnologist and oncologist",
    "oncologist",
    "veterinarian and animal behaviorist",
    "veterinarian and equine specialist",
    "veterinarian and parasitologist",
    "veterinarian",
    "herpetologist",
    "fitness and nutritional expert",
    "biology and nutrition expert",
    "nutritionist",
    "palaeontologist and palaeo",
    "vertebrae palaeontologist",
    "palaeontologist",
]

business_farming = [
    "stamp printer",
    "estranged wife of billionaire investment guru Warren Buffett",
    "billionaire advertiser and hotel developer",
    "billionaire mining tycoon",
    "billionaire advertiser",
    "billionaire and",
    "billionaire",
]
academia_humanities = [
    "Professor and Mother of Kanye West",
    "Professor of Sanskrit",
    "Professor of",
    "Egyptologist",
    "intellectual and polymath",
    "Jewish ethnographer",  # before spiritual
]
law_enf_military_operator = [
    "combatant",
    "Jewish Parachutists of Mandate member",  # before spiritual
]
spiritual = [
    "evangelical Christian and founder of Campus Crusade for Christ",
    "Christian leader and monk",
    "Christian worship renewal",
    "evangelical Christian",
    "and Christian leader",
    "Christian apologist",  # before sports
    "Christian counselor",
    "Christian Doctrine",
    "Christian preacher",
    "Christian  leader",
    "Christian leader",
    "Christianity",
    "Christian",
    "spiritual leader of the Ahmadiyya Muslim movement",
    "self styled Muslim cleric and",
    "leader of the Muslim Brotherhood",
    "Muslim preacher and polygamist",
    "Muslim community leader",
    "Black Muslim  leader",
    "Shiite Muslim cleric",
    "Shi'ite Muslim marja",
    "Muslim Sufi leader",
    "Muslim  preacher",
    "Muslim preacher",
    "Muslim leader",
    "Muslim cleric",
    "Muslim",
    "Catholic Cardinal and Grand Master Emeritus of the Equestrian Order of the Holy Sepulchre of",
    "Catholic Bishop and advocate of liberation theology",
    "Patriarch of the Chaldean Catholic Church from to",
    "Catholicos Patriarch of the Church of the East",
    "founder of Catholic journal L Brent Bozell Jr",
    "Catholic Bishop of Maitland Newcastle",
    "head of the True Catholic Church",
    "Cardinal of the Catholic Church",
    "cardinal of the Catholic Church",
    "Cardinal of the Catholic church",
    "Syro Malabar Catholic hierarch",
    "Bishop of the Catholic Church",
    "Catholic Bishop of Wilmington",
    "Catholic Bishop of Santa Rosa",
    "Catholic Bishop of Hyderabad",
    "Catholic Bishop of Nashville",
    "Ruthenian Catholic hierarch",
    "Catholic Bishop of Shanghai",
    "Traditional Catholic Bishop",
    "Maronite Catholic hierarch",
    "Chaldean Catholic hierarch",
    "Catholic fraternity leader",
    "Melkite Catholic hierarch",
    "Eastern Catholic hierarch",
    "Catholic Bishop of Bauchi",
    "Catholic Bishop of Gallup",
    "Catholic Bishop of Albany",
    "Catholic Bishop of Masaka",
    "Byzantine Catholic friar",
    "Maronite Catholic eparch",
    "traditionalist Catholic",
    "c Catholic hierarch",
    "Catholic liturgical",
    "Catholic Bishop of",
    "Catholic monsignor",
    "Catholic hierarch",
    "Catholic Cardinal",
    "Catholic official",
    "Catholic Prelate",
    "Catholic pilgrim",
    "Catholic Church",
    "Catholic Bishop",
    "Catholic leader",
    "Catholic deacon",
    "Catholic layman",
    "Catholic laity",
    "Catholic abbot",
    "Catholic monk",
    "Catholic seer",
    "Catholicism",
    "Catholic",
    "astrologer",
    "Jewish chaplain at liberation of Bergen Belsen",
    "Orthodox Jewish",
    "Jewish",
]
social = [
    'of abusive child labour in "',
    "co founder of Betty Ford Center",
    "animal welfare advocate",
    "convert to the",
    "president and co founder of the Gathering of Jewish Holocaust Survivors",  # before spiritual
    "Jewish community leader",
]
crime = [
    "hostage taker",
    "Jewish woman and Gestapo collaborator during WorldWar II",  # before spiritual
]
event_record_other = []
other_species = []

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

In [319]:
# Hard-coding info_2 for entries to not misclassify in spiritual
index = df[df["link"] == "https://en.wikipedia.org/wiki/Herbert_Freudenberger"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Leon_Uris"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Amnon_Netzer"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Liliane_Atlan"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Israel_Getzler"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Eva_J._Engel"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/Shalom_Yoran"].index
df.loc[index, "info_2"] = ""

index = df[df["link"] == "https://en.wikipedia.org/wiki/David_Cesarani"].index
df.loc[index, "info_2"] = ""

<IPython.core.display.Javascript object>

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [338]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sciences": sciences,
    "spiritual": spiritual,
    "sports": sports,
}

<IPython.core.display.Javascript object>

#### Extracting Category from `info_2`

In [340]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

CPU times: total: 3min 22s
Wall time: 3min 22s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
29006,9,John Quade,", 71, American character actor .",https://en.wikipedia.org/wiki/John_Quade,5,2009,August,",",None,,None,None,None,None,None,None,None,None,None,71.0,None,United States of America,None,",",1.791759,0,0,0,0,0,1,0,0,0,0,0,0,1
20066,27,Tungia Baker,", 64, New Zealand Māori actress and artist.",https://en.wikipedia.org/wiki/Tungia_Baker,51,2005,July,,None,,None,None,None,None,None,None,None,None,None,64.0,None,New Zealand,None,,3.951244,0,0,0,0,0,1,0,0,0,0,0,0,1


<IPython.core.display.Javascript object>

#### Checking the Number of Rows without a First Category

In [341]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

There are 5175 entries without any known_for category.


<IPython.core.display.Javascript object>

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Finding `known_for` Roles in `info_2`

In [343]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

<IPython.core.display.Javascript object>

In [501]:
# Code to check each value
roles_list.pop()

'prosecutor'

<IPython.core.display.Javascript object>

In [506]:
# Create specific_roles_list for above popped value
specific_roles_list = (
    df.loc[
        [
            index
            for index in df[df["info_2"].notna()].index
            if "alleged child sex offender" in df.loc[index, "info_2"]
        ],
        "info_2",
    ]
    .value_counts()
    .index.tolist()
)

<IPython.core.display.Javascript object>

In [507]:
# Viewing list sorted by descending length to copy to dictionary below and screen values
sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

['federal prosecutor and alleged child sex offender']

<IPython.core.display.Javascript object>

In [505]:
# Example code to quick-check a specific entry
df[df["info_2"] == "federal prosecutor and alleged child sex offender"]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy,log_num_references,sciences,social,spiritual,academia_humanities,business_farming,arts,sports,law_enf_military_operator,politics_govt_law,crime,event_record_other,other_species,num_categories
24132,5,John Atchison,", 53, American federal prosecutor and alleged child sex offender, suicide.",https://en.wikipedia.org/wiki/John_Atchison,15,2007,October,None,None,federal prosecutor and alleged child sex offender,suicide,None,None,None,None,None,None,None,None,53.0,None,United States of America,None,None,2.772589,0,0,0,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

#### Creating Lists for Each `known_for` Category

In [504]:
# Creating lists for each category
politics_govt_law = [
    "Al Qaeda official",
    "advocate for disabled people",
    "campaigner for the medical use of cannabis",  # before sciences
    "revolutionary and",
    "revolutionary",
    "physical fitness advocate",
    "victim of videotaped police beating that sparked the riots",
    "urbanist",
    "urban",
    "town",
    "city",
    "simultaneous interpreter",
    "neo",
    "breast cancer advocate",
    "Law Lord and first chairman of the Committee on Standards in Public Life",
    "chairman of the Presidential Advisory Committee on the Arts",
    "chairman of the Crow Nation since",
    
    "chief prosecutor at the Nuremberg Trials",
    "prosecutor and gun control advocate",
    "federal prosecutor",
    "public prosecutor",
    "prosecutor",
]

arts = [
    "stage hypnotist and illusionist",
    "stage manager",
    "stage and",
    "stage",
    "trombone player for Suburban Legends",  # before politics_govt_law
    "event planner",
    "planner",
    "woman believed to be the world oldest blogger",
    "and blogger",
    "blogger and subject of",
    "blogger and influencer",
    "journalism  blogger",
    "blogger in",
    "blogger",
    "string band fiddler and mandolinist and",
    "old time fiddler and banjo player",
    "bluegrass fiddler",
    "Cajun fiddler",
    "fiddler",
    "record label chairman",
    "chairman of ABC",
]
sports = [
    "fastest centenarian over metres",  # before event_record_other
    "rock climber and",
    "mountain climber",
    "sport climber",
    "rock climber",
    "free climber",
    "climber",
    "All Girls Professional Baseball League player",
    "former Baseball pitcher and pitching coach",
    "Baseball pitcher and pitching coach",
    "Baseball pitcher and umpire",
    "Baseball All Star pitcher",
    "former Baseball All Star",
    "former Baseball pitcher",
    "former Baseball umpire",
    "Baseball Hall of Famer",
    "Baseball commissioner",
    "Baseball team owner",
    "umpire in Baseball",
    "Baseball infielder",
    "Baseball pitcher",
    "Baseball catcher",
    "Baseball manager",
    "Baseball umpire",
    "hurdler and track coach",
    "sprint hurdler",
    "hurdler",
    "former chairman of S L Benfica",
    "chairman of the AFL Commission",
    "longtime chairman of F C",
]
sciences = [
    "medical examiner and Shroud of Turin investigator",
    "and medical aid developer",
    "medical  administrator",
    "medical hypnotist",
    "medical examiner",
    "medical pioneer",
    "medical",
    "pioneer in laser technology",
    "technology pioneer",
    "biotechnology",
    "on technology",
    "technology",
]

business_farming = [
    "homesteading leader",
    "CEO and chairman of United Technologies Corporation",
    "chairman of the Volkswagen automobile company",
    "co founder and former chairman of Amway",
    "chairman of the illy coffee company",
    "chairman and CEO of General Motors",
    "marine services company chairman",
    "chairman of Alliance & Leicester",
    "chairman of the Stock Exchange",
    "chairman of Tobacco Company",
    "chairman of Hunt Petroleum",
    "chairman of Instruments",
    "chairman of Metlife",
]
academia_humanities = []
law_enf_military_operator = [
    "hostage negotiator",  # before arts
    "and former police chief",
    "federal police anti drug coordinator",
    "police chief and guerilla commander",
    "chief of police of San Francisco",
    "Francoist police inspector",
    "NYPD police detective and",
    "police radio dispatcher",
    "police superintendent",
    "police commissioner",
    "senior policewoman",
    "police detective",
    "police informant",
    "police commander",
    "police inspector",
    "police official",
    "police sergeant",
    "police advisor",
    "police chief",
    "policeman",
    "police",
    "transport planner",  # before arts
]
spiritual = [
    "cleric at the Red Mosque in Islamabad",
    "Twelver Shi'a cleric",
    "Anglican cleric",
    "Shiite cleric",
    "and cleric",
    "cleric and",
    "cleric",
    "spiritual leader and head of the Sikh Dharma in the western hemisphere",
    "Chief Rabbi and spiritual leader of the Republic of from to",
    "spiritual leader and founder of Hamas",
    "Gaudiya Vaishnava spiritual leader",
    "Dvaita Vedanta spiritual leader",
    "yogi and spiritual leader",
    "Mi'kmaq spiritual leader",
    "hippy spiritual leader",
    "Hindu spiritual leader",
    "spiritual leader",
    "neocharismatic preacher",  # before politics_govt_law
    "neopagan",
]
social = [
    "healthcare and",
]
crime = [
    "organized crime operative",
    "Salt Lake City spree killer",
    "spree killer and",
    "and spree killer",
    "spree killer",
    'alleged child sex offender'
]
event_record_other = [
    "anencephalic baby who became the center of a medical controversy",  # before sciences
    "medical cannabidiol patient and reform figure",
    "medical research subject",  # before sciences
    "medical patient",
    "hostage killed in",  # before arts
    "Taliban hostage",
    "hostage in",
    "hostage",
    "centenarian and assault victim",
    "centenarian",
    "pedestrian allegedly assaulted by police at G summit protests",  # before politics_govt_law
    "man who died in police custody",
    "victim of police shooting",
    "police detainee",
    "police reformer",
    "police suspect",
    "police victim",
]
other_species = [
    "Thoroughbred hurdler",
]

<IPython.core.display.Javascript object>

In [ ]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

In [342]:
print("dunzo!")

# Sound notification when cell executes
chime.success()

dunzo!


<IPython.core.display.Javascript object>

#### Finding `known_for` Roles in `info_2`

In [ ]:
# Obtaining values for column and their counts
roles_list = df["info_2"].value_counts(ascending=True).index.tolist()

In [ ]:
# Code to check each value
roles_list.pop()

In [ ]:
# # Create specific_roles_list for above popped value
# specific_roles_list = (
#     df.loc[
#         [
#             index
#             for index in df[df["info_2"].notna()].index
#             if "general" in df.loc[index, "info_2"]
#         ],
#         "info_2",
#     ]
#     .value_counts()
#     .index.tolist()
# )

In [ ]:
# # Viewing list sorted by descending length to copy to dictionary below and screen values
# sorted(specific_roles_list, key=lambda x: len(x), reverse=True)

In [ ]:
# # Example code to quick-check a specific entry
# df[df["info_2"] == "outlaw country music singer songwriter"]

#### Creating Lists for Each `known_for` Category

In [ ]:
# Creating lists for each category
politics_govt_law = []

arts = []
sports = [


]
sciences = []

business_farming = []
academia_humanities = []
law_enf_military_operator = []
spiritual = []
social = []
crime = []
event_record_other = []
other_species = []

In [ ]:
# # Example code to quickly sort list in correct descending length search order to copy to dictionary
# temp = sorted(list(set(law_enf_military_operator)), key=lambda x: len(x), reverse=True)
# temp

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Lists of Values

In [ ]:
# Combining separate lists into one dictionary
known_for_dict = {
    "social": social,
    "spiritual": spiritual,
    "academia_humanities": academia_humanities,
    "business_farming": business_farming,
    "sciences": sciences,
    "politics_govt_law": politics_govt_law,
    "law_enf_military_operator": law_enf_military_operator,
    "crime": crime,
    "event_record_other": event_record_other,
    "other_species": other_species,
    "arts": arts,
    "sports": sports,
}

#### Extracting Category from `info_2`

In [ ]:
%%time

# Dictionary version
search_dict = known_for_dict

# Column to check
column = 'info_2'

# Dataframe
dataframe = df[column].notna()

# For loop to find role in column and extract it as category
for category, category_lst in search_dict.items():
    for role in category_lst:
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, category] = 1
                    df.loc[index, column] = item.replace(role, '').strip()

# Updating num_categories
df["num_categories"] = df[known_for_dict.keys()].sum(axis=1)

# Checking a sample of rows
df[df['num_categories']!=0].sample(2)

#### Checking the Number of Rows without a First Category

In [ ]:
# Checking the number of rows without a first category
print(
    f'There are {len(df[df["num_categories"]==0])} entries without any known_for category.'
)

#### Observations:
- We will proceed to rebuild `known_for_dict` for the next iteration.

#### Observations:
- It is time to export our dataframe and start a new notebook.

### Exporting Dataset to SQLite Database [wp_life_expect_clean12.db]()

In [ ]:
# # Exporting dataframe

# # Saving dataset in a SQLite database
# conn = sql.connect("wp_life_expect_clean12.db")
# df.to_sql("wp_life_expect_clean12", conn, index=False)

# # Chime notification when cell executes
# chime.success()

# [Proceed to Data Cleaning Part 13]()